In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json 

### Загрузка необходимых данных 

In [4]:
with open("data/test_object_lu.json", "r") as file:
    plan = json.load(file)
    
partition = cv2.imread('data/partition.jpg')
v_corners = np.load("data/v_corners.npy")
h_corners = np.load("data/h_corners.npy")
panorama = cv2.imread("data/image_1024_aligned_rgb.png")
depth = np.load("data/depth_map.npy") * 1000 # in millimeters 

In [123]:
print(depth.shape)
print(depth.max())
print(depth)
# Нормализация данных с камеры глубины
scaled_depth = ((depth - np.min(depth)) / (np.max(depth) - np.min(depth)) * 255).astype(np.uint8)
cv2.imshow('Depth Image', scaled_depth)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 1024)
6101.31
[[1214.8892 1214.8892 1214.8892 ... 1214.8892 1214.8892 1214.8892]
 [1214.9348 1214.9348 1214.9348 ... 1214.9348 1214.9348 1214.9348]
 [1215.0264 1215.0264 1215.0264 ... 1215.0264 1215.0264 1215.0264]
 ...
 [1600.1882 1600.1882 1600.1882 ... 1600.1882 1600.1882 1600.1882]
 [1600.0677 1600.0677 1600.0677 ... 1600.0677 1600.0677 1600.0677]
 [1600.0076 1600.0076 1600.0076 ... 1600.0076 1600.0076 1600.0076]]


In [4]:
# Загрузить изображения
alpha = 0.7

# Преобразовать глубину в трехканальное изображение
depth_colored = cv2.applyColorMap((scaled_depth * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Наложить изображения
overlay = cv2.addWeighted(panorama, alpha, depth_colored, 1 - alpha, 0)

# Вывести результат
cv2.imshow('Overlay', overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
cv2.imshow("source_image", panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(v_corners)

[[  30.914364  222.41164 ]
 [  30.914364  298.64883 ]
 [ 363.15613   140.81924 ]
 [ 363.15613   392.54047 ]
 [ 672.78357   149.66058 ]
 [ 672.78357   383.4991  ]
 [ 801.5895    137.09825 ]
 [ 801.5895    396.26495 ]
 [ 845.40314    76.09795 ]
 [ 845.40314   451.50623 ]
 [1008.1279    221.95033 ]
 [1008.1279    299.23853 ]]


In [7]:
start = int(v_corners[0][0])
print("start of the line:", start)
end = int(v_corners[2][0])
print("end of the line:", end)
line = []
for x in range(start, end+1, 32):
    line.append([x, h_corners[:, x][0]])
print(line)

start of the line: 30
end of the line: 363
[[30, 222.71733], [62, 192.55711], [94, 169.15553], [126, 150.01512], [158, 136.72841], [190, 129.05185], [222, 124.01126], [254, 122.649155], [286, 123.07381], [318, 126.99274], [350, 135.25735]]


In [8]:
for i in range(0, len(v_corners) - 2, 2):
    print(v_corners[i], v_corners[i + 1], v_corners[i + 2], v_corners[i + 3])

[ 30.914364 222.41164 ] [ 30.914364 298.64883 ] [363.15613 140.81924] [363.15613 392.54047]
[363.15613 140.81924] [363.15613 392.54047] [672.78357 149.66058] [672.78357 383.4991 ]
[672.78357 149.66058] [672.78357 383.4991 ] [801.5895  137.09825] [801.5895  396.26495]
[801.5895  137.09825] [801.5895  396.26495] [845.40314  76.09795] [845.40314 451.50623]
[845.40314  76.09795] [845.40314 451.50623] [1008.1279   221.95033] [1008.1279   299.23853]


In [9]:
print(h_corners)

[[221.67708 221.43863 221.38431 ... 222.38773 223.78665 223.0257 ]
 [299.86584 298.79175 299.72705 ... 298.76703 297.46896 299.3693 ]]


In [10]:
print(panorama.shape)

(512, 1024, 3)


In [181]:
panorama_copy = panorama.copy()
for pt in v_corners:
        cv2.circle(panorama_copy, (int(pt[0]), int(pt[1])), 5, (0, 0, 255), -1)
cv2.imshow("source_image", panorama_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [85]:
ceil_height = 250
area_mappings = {
    '5': (v_corners[0:4], 602),
    '0': (v_corners[2:6], 205),
    '1': (v_corners[4:8], 164),
    '2': (v_corners[6:10], 40),
    '3': (v_corners[8:12], 438),
    '4': (np.concatenate((v_corners[10:], v_corners[:2])), 165),
    'floor': ([v_corners[i] for i in range(0, 12, 2)]),
    'ceil': ([v_corners[i] for i in range(1, 12, 2)]),
}
print(area_mappings)

{'5': (array([[ 30.914364, 222.41164 ],
       [ 30.914364, 298.64883 ],
       [363.15613 , 140.81924 ],
       [363.15613 , 392.54047 ]], dtype=float32), 602), '0': (array([[363.15613, 140.81924],
       [363.15613, 392.54047],
       [672.78357, 149.66058],
       [672.78357, 383.4991 ]], dtype=float32), 205), '1': (array([[672.78357, 149.66058],
       [672.78357, 383.4991 ],
       [801.5895 , 137.09825],
       [801.5895 , 396.26495]], dtype=float32), 164), '2': (array([[801.5895 , 137.09825],
       [801.5895 , 396.26495],
       [845.40314,  76.09795],
       [845.40314, 451.50623]], dtype=float32), 40), '3': (array([[ 845.40314,   76.09795],
       [ 845.40314,  451.50623],
       [1008.1279 ,  221.95033],
       [1008.1279 ,  299.23853]], dtype=float32), 438), '4': (array([[1008.1279  ,  221.95033 ],
       [1008.1279  ,  299.23853 ],
       [  30.914364,  222.41164 ],
       [  30.914364,  298.64883 ]], dtype=float32), 165), 'floor': [array([ 30.914364, 222.41164 ], dtype=fl

In [204]:
import numpy as np
from e2p import e2p


def forward_gnomonic_projection(theta0, phi0, theta, phi):
    cosc = np.sin(phi0) * np.sin(phi) + np.cos(phi0) * np.cos(phi) * np.cos(theta - theta0)
    x = np.cos(phi) * np.sin(theta - theta0) / cosc
    y = (np.cos(phi0) * np.sin(phi) - np.sin(phi0) * np.cos(phi) * np.cos(theta - theta0)) / cosc
    return x, y

def e2p_coor(eq_xy, fov_deg, u_deg, v_deg, eq_hw, out_hw):
    u_rad, v_rad = np.radians(u_deg), np.radians(v_deg)
    h_fov, v_fov = np.radians(fov_deg[0]), np.radians(fov_deg[1])
    
    latitude = np.linspace(-np.pi, np.pi, num=eq_hw[1], dtype=np.float32)  # Широта 
    longitude = np.linspace(-np.pi, np.pi, num=eq_hw[0],  dtype=np.float32) / 2 # Долгота 
    grid = np.stack(np.meshgrid(latitude, longitude), axis=-1)
    
    # Перевод [x, y] в сферические координаты [phi, lambda] 
    theta, phi = grid[int(eq_xy[1]), int(eq_xy[0]), :]
    px0, py0 = forward_gnomonic_projection(u_rad, v_rad, theta, phi)
    x_max, y_max = np.tan(h_fov / 2), np.tan(v_fov / 2)
    
    # Нормализация значений точки относительно области изображения 
    px_norm, py_norm = abs(px0 + x_max) / (2 * x_max), abs(py0 - y_max) / (2 * y_max)
    # Нахождение координат точки на изображении 
    return px_norm * out_hw[1], py_norm * out_hw[0]


# area, fov_deg, u_deg, v_deg, in_rot
views = [
         ['0', (120, 120), 0, 20, 0],
         # ['1', (150, 150), 80, 0, 0],
         # ['2', (120, 150), 80, 0, 0],
         # ['3', (130, 150), 150, 0, 0],
         # ['4', (130, 150), 150, 0, 0],
         # ['5', (150, 140), -110, 0, 0]
         ]
equirectangular_img = panorama
h, w = equirectangular_img.shape[:2]

for area, fov_deg, u_deg, v_deg, in_rot in views:
    out_hw = (
        (h * fov_deg[1]) // 180,
        (w * fov_deg[0]) // 360,
    )
    pers_img = e2p(equirectangular_img, fov_deg, u_deg, v_deg, out_hw, in_rot_deg=0, mode='bilinear')
    pers_points = []
    for pt in area_mappings[area][0]:
        point = (e2p_coor(pt, fov_deg, u_deg, v_deg, (h, w), out_hw))
        cv2.circle(pers_img, (int(point[0]), int(point[1])), 2, (0, 0, 255), -1)
        print(point)
        cv2.imshow('Overlay', pers_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        pers_points.append(point)
        
    dst_hw = (250, area_mappings[area][1])
    src_points = np.array(pers_points, dtype=np.float32)
    dst_points = np.array([[0, 0], [0, 250], [dst_hw[1], 0], [dst_hw[1], 250]], dtype=np.float32)
    
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    corrected_image = cv2.warpPerspective(pers_img, perspective_matrix, (dst_hw[1], dst_hw[0]))
    
    # Сохранение изображений
    cv2.imwrite(f"views/{area}_proj.jpg", corrected_image) 

(106.35796478624614, 526.6262442031167)
(89.08673356823769, 84.29715304814377)
(490.66441815473894, 520.0810220902373)
(265.94116631531824, 84.87677921145476)


In [205]:
# area, fov_deg, u_deg, v_deg, in_rot
views = [['ceil', (130, 140), 0, 90, 0],
         ['floor', (130, 130), 0, -110, 0],
         ]
dst_points = np.array([[0, 0], [0, 602], [205, 602], [205, 438], [165, 438], [165, 0]], dtype=np.float32)
for area, fov_deg, u_deg, v_deg, in_rot in views:
    out_hw = (
        (h * fov_deg[1]) // 180,
        (w * fov_deg[0]) // 360,
    )
    pers_img = e2p(equirectangular_img, fov_deg, u_deg, v_deg, out_hw, in_rot_deg=0, mode='bilinear')
    pers_points = []
    for pt in area_mappings[area]:
        point = (e2p_coor(pt, fov_deg, u_deg, v_deg, (h, w), out_hw))
        cv2.circle(pers_img, (int(point[0]), int(point[1])), 2, (0, 0, 255), -1)
        
        cv2.imshow('Overlay', pers_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        pers_points.append(point)
        
    dst_hw = (602, 205)
    src_points = np.array(pers_points, dtype=np.float32)
    print(src_points)
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    corrected_image = cv2.warpPerspective(pers_img, perspective_matrix, (dst_hw[1], dst_hw[0]))
    
    # Сохранение изображений
    cv2.imwrite(f"views/{area}_proj.jpg", corrected_image) 

[[125.55153  67.28263]
 [123.41894 238.80429]
 [256.44736 239.12007]
 [256.42914 186.26448]
 [214.03755 186.13176]
 [213.38696  64.22364]]


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:3547: error: (-215:Assertion failed) src.checkVector(2, CV_32F) == 4 && dst.checkVector(2, CV_32F) == 4 in function 'cv::getPerspectiveTransform'


In [11]:
import open3d as o3d
import open3d.cpu.pybind.core as o3c
import numpy as np
from skimage import transform

def panoramic_to_pointcloud(image, depth_map, fov_horizontal_deg=360, fov_vertical_deg=180):
    # Параметры панорамы
    height, width, _ = image.shape
    fov_horizontal_rad = np.radians(fov_horizontal_deg)
    fov_vertical_rad = np.radians(fov_vertical_deg)

    resized_image = transform.resize(image, (height * 7, width * 7, 3), mode='reflect')
    resized_depth_map = transform.resize(depth_map, (height * 7, width * 7), mode='reflect')
    new_height, new_width, _ = resized_image.shape
    points = []
    colors = []

    # Преобразование координат из пиксельных в 3D
    for v in range(new_height):
        for u in range(new_width):
            d = resized_depth_map[v, u]
            if d > 0:
                phi = (v / new_height) * fov_vertical_rad
                theta = (u / new_width) * fov_horizontal_rad

                x = d * np.sin(phi) * np.cos(theta)
                y = d * np.sin(phi) * np.sin(theta)
                z = d * np.cos(phi)

                points.append([x, y, z])
                colors.append(list(resized_image[v, u, :]))

    # Создание PointCloud
    pcd = o3d.t.geometry.PointCloud(o3c.Tensor(points, o3c.float32))
    pcd.point.colors = o3c.Tensor(colors, o3c.float32)
    # pcd.estimate_normals(max_nn=30, radius=0.3)

    return pcd

# Пример использования
# image - изображение с панорамой
# depth_map - карта глубины для каждого пикселя панорамы
# fov_horizontal_deg, fov_vertical_deg - горизонтальное и вертикальное поле зрения
pointcloud = panoramic_to_pointcloud(panorama, depth)
print(pointcloud)


PointCloud on CPU:0 [25690112 points (Float32)].
Attributes: colors (dtype = Float32, shape = {25690112, 3}).


In [122]:
o3d.visualization.draw_geometries([pointcloud.to_legacy()],
                                  point_show_normal=True)

In [18]:
o3d.io.write_point_cloud("pointclouds/mesh7x.ply", pointcloud.to_legacy())

True

In [ ]:
pcd = pointcloud.to_legacy()
triangles = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha=0.5)

# Визуализация результатов
o3d.visualization.draw_geometries([pcd, triangles])

In [ ]:
# alpha shape 
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha=100)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)